# Post Processing Analysis of 3D Subduction Simulations with Plotly

#This script aim to use the output of the numerical Velocity field to extract the evolution of the vorticity field through time, for comparison with the analytical work on O'Connell 1992.

-------
RM- $\eta_M/ \eta_{WL} = 10^{0}$
![alt](testrm.gif) 

Case 1: this script aims to plot the value of the vorticity field in three different plane at the appropriate depth at a time step.
------
To characterize the induced flow dynamics we calculate the horizontal divergence and the vertical component of vorticity as:

\begin{equation*}
  \nabla_{h}=  \left( \frac{\partial}{\partial x},\frac{\partial}{\partial y} \right) \cdot \textbf{u}, \quad w_{z}=  \hat{z} \cdot \nabla \times \textbf{u}
\end{equation*}

In [ ]:
import h5py as h5
import matplotlib.pyplot as plt
import numpy as np

import underworld as uw
import math
from underworld import function as fn

import os

import matplotlib.pyplot as pyplot

from mpl_toolkits.mplot3d import Axes3D

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import matplotlib.pyplot as plt

from scipy.signal import butter, lfilter, freqz, savgol_filter
from scipy.signal import wiener, medfilt
from matplotlib.colors import Normalize

import matplotlib.colors as colors
import matplotlib.cbook as cbook 

import plotly.graph_objects as go
import pandas as pd

In [ ]:
#==========================================================================
# Initial parameters for post-processing analysis
#==========================================================================
# Specify the directory in which you want to visualize the models
outputPath = os.path.join(os.path.abspath("."),"PostProcAnalyisis3drm/")

if uw.mpi.rank==0:
    if not os.path.exists(outputPath):
        os.makedirs(outputPath)
uw.mpi.barrier()

# Input parameters
printmod=1;             # Want the figures to be saved?
FiguresVisible = 'on';  # want to see the figures?

# Base directory
main_directory    = ('/home/jovyan/3D-Post_Processing/')

# Directory you wish to analise
ndirs             = [1]

# Get to the base directory
os.chdir(main_directory)
cur_dir           = os.getcwd()
print ('Current directory: ',cur_dir)
plt.figure().clf()

print ('-----------------------------')
print ('Start sampling the data:')

In [ ]:
#==========================================================================
# Simulation details
#==========================================================================
#==========================================================================
# Resolution
#==========================================================================
xRes      =  256
zRes      =  64
yRes      =  64
time         = 157.              # Initial time
#==========================================================================
# Characteristics Values          #
#==========================================================================
l_D            =   1.e5        #   [m] 100km
eta_D          =   1.e21       #   [Pa s]
sigma_D        =   1.e10       #   [Pa] 10 GPa ~ 250 km depth
Temperature_D  =   1000       #   [K]


#==========================================================================
# Reference choosen values
#==========================================================================
l_DL            =   1.
eta_DL          =   1.
sigma_DL        =   1.
Temperature_DL  =   1.


#==========================================================================
# Trivial Scaling Factor
#==========================================================================
l_SF            =   l_DL     / l_D
eta_SF          =   eta_DL   / eta_D
sigma_SF        =   sigma_DL / sigma_D
Temperature_SF  =   Temperature_DL / Temperature_D

#==========================================================================
# Basic scaling coefficient LAMEM
#==========================================================================
time_SF         =   eta_SF/sigma_SF
force_SF        =   sigma_SF*(l_SF*l_SF)
g_SF            =   l_SF/time_SF/time_SF
m_SF            =   force_SF/g_SF

#==========================================================================
# Other scaling coefficients
#==========================================================================
angle_SF           = math.pi/180;
area_SF            = l_SF*l_SF
volume_SF          = area_SF*l_SF
energy_SF          = force_SF*l_SF
power_SF           = energy_SF/time_SF
v_SF               = l_SF/time_SF
strain_rate_SF     = 1.0/time_SF
rho_SF             = m_SF/volume_SF

#==========================================================================
# Thermal scaling coefficients
#==========================================================================
Thermal_conductivity_SF         = power_SF/l_SF /Temperature_SF
Thermal_expansion_SF            = 1/Temperature_SF
capacity_SF                     = energy_SF/ Temperature_SF / m_SF
Thermal_diffusivity_SF          = (l_SF*l_SF)  /  time_SF #in uw2

#==========================================================================
# Additional UNITS
#==========================================================================
yr     = 3600.0*24.0*365.25;
Myr    = 1e6*yr;

km     = 1e3;
cm     = 1.e-2;
cm_yr  = cm/yr;
MPa    = 1e6;
mW     = 1e-3;
K      =  273.15
#==========================================================================
# Dimensional physical data
#==========================================================================
DII_ref        =   1e-16
boxWidth       =    3000.0*km
boxHeight      =    660.0*km
boxLength      =    4000.0*km
boxHeight      =    660.0*km
l_max_op       =    1750.0*km
l_min_op       =    - 1000.0*km

Depth_slab     =    - 256.2536*km
l_op           =    - 310.0*km

#==========================================================================
# Getting dimension-less values
#==========================================================================

boxWidth_DL      =  boxWidth * l_SF
boxLength_DL     =  boxLength * l_SF
boxHeight_DL     =  boxHeight * l_SF
l_max_op_DL      =  l_max_op * l_SF
l_min_op_DL      =  l_min_op * l_SF
Depth_slab_DL    =  Depth_slab * l_SF
l_op_DL          =  l_op * l_SF

dim       =  3

L         =  boxLength_DL
W         =  boxWidth_DL
H         =  boxHeight_DL

dx         =   boxLength_DL/xRes
dz         =   boxHeight_DL/zRes
dy         =   W/yRes
dy         =     W/yRes

y_back     =    -W/2;                #  coord of the front margin
w_min_op   =    -W               # attached to the front wall
w_max_wl   =    -W/2 + W/2;
w_max_slab =    -y_back
x_left     =   -L/2                 #  coord of the left margin
z_bot      =   -H                   #  coord of the bottom margin
z_surface  =   -H+H
#modelTime = str("{0:.5f}").format(float(time)/ time_SF/ Myr/10000 )

gridpoints_x = xRes+1
gridpoints_z = zRes+1
gridpoints_y = yRes+1
gridpoints_xy=gridpoints_x*gridpoints_y
gridpoints_xz=gridpoints_x*gridpoints_z

In [ ]:
#==========================================================================
# INITIALIZE MESH AND MESH VARIABLES
#==========================================================================
mesh = uw.mesh.FeMesh_Cartesian( elementType = ("Q1/dQ0"),
                                elementRes  = (xRes, yRes, zRes),
                                minCoord    = (x_left , z_bot, y_back),
                                maxCoord    = (-x_left, z_surface, -y_back) )

velocityField       = uw.mesh.MeshVariable( mesh=mesh,         nodeDofCount=dim )
pressureField       = uw.mesh.MeshVariable( mesh=mesh.subMesh, nodeDofCount=1 )
temperatureField    = uw.mesh.MeshVariable( mesh=mesh, nodeDofCount=1 )

In [ ]:
c = 0
time_sol_perdir = []
steps_per_dir = []

# 1st loop over the different simulations directories
for i in ndirs:   
    print (i)
    sim_directory  = ('m'+ str(ndirs[c]))    
    sim_path       = main_directory +sim_directory      

    output_path  = sim_path
    os.chdir(output_path)
       
    # 1st if outputdir exisit
    if  output_path: 
        
        f = h5.File("vel-0.00048.h5", "r")
        if f:
            print ("found it")
        # Get and print list of datasets within the H5 file
            datasetNames = [n for n in f.keys()]
            for n in datasetNames:
                print(n)

In [ ]:
#==========================================================================
# get some scaling done
#==========================================================================
def lenkm(l):
    myarray = np.array(l)
    lengthDim = []
    lengthDim = [np.array(i) /l_SF /km  for i in l]
    return lengthDim

def timeMyr(t):
    myarray = np.array(t)
    timeDim = []
    timeDim[:] = [ np.array(i) / time_SF/ Myr for i in myarray]   
    return timeDim

def VelcmYr(v):
    myarray = np.array(v)
    VelDim = []
    VelDim = [np.array(i) /v_SF* (yr/cm) for i in myarray]
    return VelDim

def strain_fn(minus_s):
    myarray = np.array(minus_s)
    VelDim = []
    VelDim = [np.array(i) /strain_rate_SF for i in myarray]
    return VelDim

Berc_SF = 1e9*yr;

def Bercov(values):
    myarray = np.array(values)
    BercApprox= []
    BercApprox = [np.array(i) *(time_SF* Berc_SF) for i in myarray]
    return BercApprox

def returnarrayoffloats(DL_list):
    myarray = np.array(DL_list)
    DL_array = []
    DL_array = [np.array(i) for i in myarray]
    return DL_array

In [ ]:
# load the velocity field from the h5 file
print (mesh.elementRes)
print (f.filename)
vel = mesh.add_variable(3)
vel.load(str(f.filename))

# the underworld (uw) function from (Moresi, 2003) allow us to extract gradients and interpolatie functions at the mesh nodes in an MPI friendly env

In [ ]:
# projects the gradient of the velocity field on the mesh nodes to decrease numerical noise
projected_fn_gradient = mesh.add_variable(9)
gradient_projector = uw.utils.MeshVariable_Projection(projected_fn_gradient,vel.fn_gradient,type=0)
gradient_projector.solve()

In [ ]:
#extract vorticty and divergence from the gradient function -do note that the cartesian plane is oriented as x-z-y
w_z                 =   (projected_fn_gradient[6] - projected_fn_gradient[2])
div_h               =  (projected_fn_gradient[0] - projected_fn_gradient[8]) 

In [ ]:
# Create surfaces to integrate the div or vorticity fields at different depths
velPoints_z_x      =        np.linspace(x_left, -x_left, gridpoints_x)
velPoints_z_y      =        np.linspace(-y_back, (y_back+0.2), gridpoints_y)

X                  =        velPoints_z_x
Y                  =        velPoints_z_y
x,y                =        np.meshgrid (X,Y)

d                  = x.ravel()
e                  = y.ravel()
f                  = d.reshape( ((gridpoints_xy), 1))
reversed_arr       = e[::-1]
g                  = reversed_arr.reshape( ((gridpoints_xy), 1))
i                  = np.hstack((f ,g))

#depths are taken 2-4 dz away from phase boundaries to avoid too much numerical noise
zUc1_array          = np.array((gridpoints_xy) * ([  -(dz*28)]))
vel_points_zUC1     = np.insert(i, 1, values=zUc1_array, axis=1)

zSurface_array      = np.array((gridpoints_xy) * ([  z_surface -(dz*4)]))
vel_points_zSurface = np.insert(i, 1, values=zSurface_array, axis=1)

zbot_array          = np.array((gridpoints_xy) * ([  z_bot+0.3]))
vel_points_zbot     = np.insert(i, 1, values=zbot_array, axis=1)

# The vorticity field is integrated only at specific depths in the mesh volume.

In [ ]:
# evaluate vorticity at different surface plans
evaluated_w_zsurface   =  w_z.evaluate_global(vel_points_zSurface)
w_zsurface_sol        = Bercov(evaluated_w_zsurface )

evaluated_w_zUC1       =  w_z.evaluate_global(vel_points_zUC1)
w_zUC1_sol             = Bercov(evaluated_w_zUC1 )

evaluated_w_zbot      =  w_z.evaluate_global(vel_points_zbot)
w_zbot_sol             = Bercov(evaluated_w_zbot)

# Filtering of the data

In [ ]:
# filter the data and grab coords values in k

w_zbot_sol1    = np.array(w_zbot_sol , dtype=float)
w_zbot_sol_re  = w_zbot_sol1.reshape((w_zbot_sol1.shape[0],))

w_med_botkm    =medfilt(w_zbot_sol_re,kernel_size=25)
n = 10 # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1
w_med_lf_botkm = lfilter(b,a,w_med_botkm)

# below vorticity filtered at the 660km 
w_zUC1_sol1=np.array(w_zUC1_sol , dtype=float)
w_zUC1_sol_re=w_zUC1_sol1.reshape((w_zUC1_sol1.shape[0],))

w_med_zUC1km    =medfilt(w_zUC1_sol_re,kernel_size=25)
n = 10 # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1
w_med_lf_zUC1km = lfilter(b,a,w_med_zUC1km  )

# filter the data and grab coords values in km
w_z_surface1=np.array(w_zsurface_sol , dtype=float)
w_z_surface_sol_re=w_z_surface1.reshape((w_z_surface1.shape[0],))

# Apply first filter vorticity surface plane
w_med=medfilt(w_z_surface_sol_re,kernel_size=25)
n = 10 # the larger n is, the smoother curve will be
b = [1.0 / n] * n
a = 1
# Apply second filter
w_med_lf_surfacekm = lfilter(b,a,w_med)

velPoints_z_surfacekm=vel_points_zSurface/l_SF/km 
X = velPoints_z_surfacekm[:,0]
Y = velPoints_z_surfacekm[:,2]
X = np.array(X, dtype=float)
Y = np.array(Y, dtype=float)

X_surfacekm = X.reshape((X.shape[0],))
Y_surfacekm= Y.reshape((Y.shape[0],))

# Pandas dataframe converts data to the righ format for surface plotting, such as that z2, z3 and z4 are the required vorticity surfaces.

In [ ]:
# change shape of the data for plotting (?) #surface div
xyz = {'x': X_surfacekm, 'y': Y_surfacekm, 'z':w_med_lf_surfacekm }

# put the data into a pandas DataFrame (this is what my data looks like)
df = pd.DataFrame(xyz, index=range(len(xyz['x'])))
x1 = np.linspace(df['x'].min(), df['x'].max(), len(df['x'].unique()))
y1 = np.linspace(df['y'].min(), df['y'].max(), len(df['y'].unique()))
x2, y2 = np.meshgrid(x1, y1)
z2 = griddata((df['x'], df['y']), df['z'], (x2, y2), method='cubic')

# change shape of the data for plotting (?) # lithosphere base div
xyz = {'x': X_surfacekm, 'y': Y_surfacekm, 'z':w_med_lf_zUC1km }

# put the data into a pandas DataFrame (this is what my data looks like)
df = pd.DataFrame(xyz, index=range(len(xyz['x'])))
x1 = np.linspace(df['x'].min(), df['x'].max(), len(df['x'].unique()))
y1 = np.linspace(df['y'].min(), df['y'].max(), len(df['y'].unique()))
x2, y2 = np.meshgrid(x1, y1)
z3 = griddata((df['x'], df['y']), df['z'], (x2, y2), method='cubic')

# change shape of the data for plotting (?) # 660km
xyz = {'x': X_surfacekm, 'y': Y_surfacekm, 'z':w_med_lf_botkm}

# put the data into a pandas DataFrame (this is what my data looks like)
df = pd.DataFrame(xyz, index=range(len(xyz['x'])))
x1 = np.linspace(df['x'].min(), df['x'].max(), len(df['x'].unique()))
y1 = np.linspace(df['y'].min(), df['y'].max(), len(df['y'].unique()))
x2, y2 = np.meshgrid(x1, y1)
z4 = griddata((df['x'], df['y']), df['z'], (x2, y2), method='cubic')


In [ ]:
mycolorscale = [[0, 'magenta'],[0.3, 'violet'], [0.35, 'blue'], [0.5, 'green'],[0.6, 'yellow'],[0.8, 'orange'], [1, 'red']]


fig1 = go.Figure(data=[
        go.Surface (
        contours = {
        "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
        "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
                      }, coloraxis='coloraxis',
            
            colorbar=dict(
            title='Vorticity [Gyr-1]', # title here
            titleside='right',
            titlefont=dict(
            size=14,
            family='Arial, sans-serif'),
            len=0.8,),
            

            z=z2, opacity=0.9),

        go.Surface(contours = {
        "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
        "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
                      },  coloraxis='coloraxis',
      
                   z=z3-110,opacity=0.9, ),
        go.Surface(contours = {
        "x": {"show": True, "start": 1.5, "end": 2, "size": 0.04, "color":"white"},
        "z": {"show": True, "start": 0.5, "end": 0.8, "size": 0.05}
                      }, coloraxis='coloraxis',
                   
                   z=z4-660, opacity=0.9)])

fig1.update_layout(coloraxis = dict(colorscale=mycolorscale, cmin =-320, cmax=320, colorbar_thickness=25))


                 
fig1.update_layout(scene = dict(
                    xaxis_title='X [km]',
                    yaxis_title='Y [km]',
                    zaxis_title='Z [km]'),
                    )



fig1.show()


In [ ]:
# the above is not appropriate as the value of the function are also varied.